In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())) + "/..")
if os.getcwd().endswith("Topic2"):
    os.chdir("..")
    print(f"Changed to {os.getcwd()}")

from codebase import load_excel, load_odin_as_ml_dataset, plot_confusion_matrix, make_ml_dataset, run_binary_regression, transport_modes, trip_motives, province_codes, transport_mode_col

Changed to c:\Users\flori\OneDrive - TU Eindhoven\Master TUe 2024-2025\Design-of-AI-driven-business-operation


In [2]:
years = [2019, 2020, 2021, 2022, 2023]
dfs = []
for year in years:
    odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", f"OdiN {year}", f"ODiN{year}_Databestand.csv")
    if year in dfs:
        odin_excel_path = odin_excel_path.replace("Databestand", "Databestand_v2.0")
    df = load_excel(odin_excel_path)
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

Loaded file from CSV


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
len(df), len(df["OPID"].unique())

In [ ]:
# Here just in case
year = 2022
odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", f"OdiN {year}", f"ODiN{year}_Databestand.csv")
df = load_excel(odin_excel_path)
df = df[df["Verpl"] == 1]
len(df)

In [ ]:
len(df_all) / df_all["VertPC"].unique().shape[0]

In [ ]:
# Create a histogram for the number of trips for each transport mode and translate to 
df[transport_mode_col].value_counts().plot(kind="bar", title="Number of trips per transport mode")

In [ ]:
for val, name in transport_modes.items():
    print(f"Number of {name}: ({len(df[df[transport_mode_col] == val])})")

In [ ]:
print(transport_modes)

In [ ]:
print(trip_motives)

In [ ]:
print(province_codes)

In [ ]:
df.columns

In [ ]:
max_dist=600 # max distance in 100s of meters
motives=None 
vehicles_to_include = [1, 2, 3, 7, 8, 9, ]
for transport_mode in vehicles_to_include:
    run_binary_regression(
        df,
        test_size=0.2,
        transport_modes_predict=[transport_mode, ],
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/{motives}/{transport_modes[transport_mode].replace(r'/', ' or ')}_{max_dist}",
    )

In [ ]:
combined_modes = [[7, 8, 9]]
for tm in combined_modes:
    transport_mode_str = ", ".join([transport_modes[tm_] for tm_ in tm])
    run_binary_regression(
        df,
        test_size=0.2,
        transport_modes_predict=tm,
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/{motives}/{transport_mode_str.replace(r'/', ' or ')}_{max_dist}",
    )

Now a classifier that classifies what mode of transport was used

In [ ]:
print(transport_modes)

In [ ]:
from codebase import run_multiclass_classification, distance_col, origin_col, province_codes, level_mapping_suffix, age_col, transport_mode_col, motive_col, urbanness_col


df_multiclass = df[df[transport_mode_col] != df[transport_mode_col].max()]
motives=None
origins=None#[7, 8, ]
location_level=2
categorical_features=[origin_col]
numerical_features=[distance_col, urbanness_col, motive_col]
origin_str = ', '.join(province_codes[code] for code in origins) if origins is not None else "NL"
motive_str = ', '.join(trip_motives[motive] for motive in motives) if motives is not None else "all motives"

_ = run_multiclass_classification(
    df_multiclass,
    test_size=0.2,
    destinations=None,
    origins=origins,
    location_level=location_level,
    categorical_features=categorical_features,
    numerical_features=numerical_features,
    motives=motives,
    plot=True,
    savename=f"graphics/classification_results/multiclass/{level_mapping_suffix[location_level]}/{origins}/{motives}/{categorical_features + numerical_features}",
    plot_title=f"Confusion Matrix for {origin_str} and {motive_str}",
)

In [ ]:
from codebase import urbanness_col, motive_col

age_range = (0, np.inf)

df_less_vehicles = df_multiclass[df_multiclass[transport_mode_col].isin(vehicles_to_include)]
df_less_vehicles = df_less_vehicles[(df_less_vehicles[age_col] < age_range[1]) & (df_less_vehicles[age_col] > age_range[0])]

motives=None
origins=None#[7, 8, ]
location_level=2
categorical_features=[origin_col]
numerical_features=[distance_col, urbanness_col, motive_col]
origin_str = ', '.join(province_codes[code] for code in origins) if origins is not None else "NL"

_ = run_multiclass_classification(
    df_less_vehicles,
    test_size=0.2,
    destinations=None,
    origins=origins,
    location_level=location_level,
    categorical_features=categorical_features,
    numerical_features=numerical_features,
    motives=motives,
    plot=True,
    savename=f"graphics/classification_results/multiclass/{level_mapping_suffix[location_level]}/{origins}/less_vehicles/age_{age_range[0]}_{age_range[1]}/{categorical_features + numerical_features}",
    plot_title=f"Confusion Matrix for {origin_str} by {', '.join(transport_modes[tm] for tm in vehicles_to_include)}",
)

In [ ]:
# Include a subset af transport modes to compare the bike to

run_binary_regression(
        df_less_vehicles[df_less_vehicles[transport_mode_col].isin([1, 8,])],
        test_size=0.2,
        transport_modes_predict=[8, ],
        motives=motives,
        max_dist=max_dist,
        savename=f"graphics/classification_results/binary_regression/car_foot_and_bike/{transport_modes[transport_mode].replace(r'/', ' or ')}_{max_dist}",
    )